### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2


### Дисциплина: Информационный анализ данных

##### Студент:  Ильин Никита 
##### Группа:   НФИбд-01-19

## Москва 2022
***

### Вариант №4

Алгоритм: Apriori 

День недели (поле order_dow таблицы orders): “2” 

Код департамента (поле department_id таблицы products): “2” 

Показатель оценки ассоциативных правил: лифт (lift) 

## Задание

Для закрепленного за Вами варианта лабораторной работы:

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

2.  Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец `product_id`) в транзакциях датафрейма.

3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 

5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

7.	Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.



In [1]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('instacart.db')
type(conn)

sqlite3.Connection

In [2]:
cursor = conn.cursor()

type(cursor)

sqlite3.Cursor

2.  Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец `product_id`) в транзакциях датафрейма.



In [3]:
pd.read_sql_query("SELECT * FROM departments", conn).tail()

,department_id,department
16,17,household
17,18,babies
18,19,snacks
19,20,deli
20,21,missing


In [4]:
pd.read_sql_query("SELECT * FROM products", conn).tail()

,product_id,product_name,aisle_id,department_id
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8
49687,49688,Fresh Foaming Cleanser,73,11


In [5]:
order_products__train = pd.read_sql_query(
    """SELECT opt.order_id, opt.product_id, add_to_cart_order, reordered, product_name
        FROM order_products__train as opt, orders as ord, products as pr
        WHERE ord.order_dow=2 AND pr.department_id=2
        AND ord.order_id = opt.order_id AND pr.product_id = opt.product_id""", conn)
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2316178,32115,2,1,93/7 Ground Beef
1,988,4818,4,1,Classic Vanilla Coffee Creamer
2,245330,11537,5,0,Anti Bug Shake & Spray
3,3398797,26756,4,0,Light CocoWhip! Coconut Whipped Topping
4,1673226,5435,9,0,Melatonin 1 Mg Peppermint Sublingual Tablets


In [6]:
len(order_products__train)

202

In [7]:
len(order_products__train['product_id'].unique())

121

3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.



In [8]:
order_products__train['product_name'].value_counts()[:5]

Roasted Almond Butter                      14
Light CocoWhip! Coconut Whipped Topping     8
Roasted Unsalted Almonds                    6
Pierogi Potato & Cheese                     5
Organic Breakfast Blend Coffee              5
Name: product_name, dtype: int64

4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 



In [9]:
import itertools
dataset = order_products__train.groupby('order_id')['product_name'].apply(list).to_dict()
dataset

{'1002376': ['Natural Calm Anti-Stress Drink'],
 '1002771': ['Plain Salt'],
 '1010544': ['Candle Lighter'],
 '1026937': ['Organic Turmeric Powder'],
 '1032117': ['Tahini Sesame Paste'],
 '1033632': ['Roasted Unsalted Almonds'],
 '103529': ['Roasted Almond Butter'],
 '10362': ['Coconut Flour'],
 '1041996': ['California White Zinfandel'],
 '1045720': ['Carob Spirulina Energy Chunks'],
 '1052847': ['Butt Paste Original Diaper Rash Ointment',
  'Baby Healing Ointment'],
 '1073137': ['Cherry Vanilla Granola'],
 '1082545': ['Pierogi Potato & Cheese'],
 '1111176': ['Recolte Wine'],
 '1115459': ['Healing Baby Diaper Rash Cream'],
 '1130967': ['Roasted Almond Butter'],
 '1137096': ['Roasted Almond Butter'],
 '1162810': ['Coffee Mate French Vanilla Creamer Packets'],
 '1164706': ['Liquid Teething Relief'],
 '1183347': ['Bold & Spicy Bloody Mary Mix'],
 '1202295': ['Baby Healing Ointment'],
 '1209507': ['93/7 Ground Beef'],
 '1247960': ['Black Chia Seeds'],
 '1273758': ['Light CocoWhip! Coconut W

In [10]:
order_products__train[['order_id', 'product_name']].groupby('order_id')['product_name'].count().sort_values(ascending=False)


order_id
2776395    2
220023     2
3298705    2
2111359    2
3256008    2
          ..
2101195    1
2120211    1
216859     1
2243739    1
988        1
Name: product_name, Length: 195, dtype: int64

In [11]:
dataset.get('2776395')
dataset.get('220023')
dataset.get('3298705')

['Margarita Salt', 'Light Margarita']

5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.



In [12]:
from mlxtend.preprocessing import TransactionEncoder

dataset1 = []

for i in dataset.values():
    dataset1.append(i)
    
keyss = []

for i in dataset.keys():
    keyss.append(i)
    
te = TransactionEncoder()
dataset_bin = te.fit(dataset1).transform(dataset1)

In [13]:
df = pd.DataFrame(dataset_bin, columns=te.columns_, index=keyss)
df.head(5)

,5-HTP 100 Mg Vegetarian Capsules,93/7 Ground Beef,All Purpose Precision Tip 2 Pack,Anti Bug Shake & Spray,Aromatic Bitters,Baby Healing Ointment,Baby Powder,Baby Tummy Gripe Water Dietary Supplement,BabyRub® Soothing Ointment,Beef Chuck Roast,...,Tranquil Rose Massage Oil,Tropical Fruit Electrolyte Solution,Ultimate Intensive Healing Hand Cream,Vitamin Code Kids Cherry Berry Chewables,Vitamin Code Kids Chewable Whole Food Multivitamin For Kids Cherry Berry,Walnuts,Wellness Herbal Kids Tincture,Whispering Angel Rosé,White Zinfandel,Yogurt Covered Almonds
1002376,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1002771,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1010544,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1026937,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1032117,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
asd= []
for i in df.columns:
    asd.append((df[i]==True).sum())
pd.Series(asd,index=df.columns).sort_values(ascending=False)[:5]

Roasted Almond Butter                      14
Light CocoWhip! Coconut Whipped Topping     8
Roasted Unsalted Almonds                    6
Pierogi Potato & Cheese                     5
Organic Breakfast Blend Coffee              5
dtype: int64

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).



In [15]:
from mlxtend.frequent_patterns import apriori

apr = apriori(df, min_support=1/df.shape[0],use_colnames=True)
apr

,support,itemsets
0,0.005128,(5-HTP 100 Mg Vegetarian Capsules)
1,0.015385,(93/7 Ground Beef)
2,0.005128,(All Purpose Precision Tip 2 Pack)
3,0.005128,(Anti Bug Shake & Spray)
4,0.005128,(Aromatic Bitters)
...,...,...
123,0.005128,"(Cherry Vanilla Granola, Carob Spirulina Energ..."
124,0.005128,"(Cuticle Rehab, Max AAA Batteries)"
125,0.005128,"(Italian Sweet Creme Creamer, Rainbow Roll)"
126,0.005128,"(Light Margarita, Margarita Salt)"


7.	Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.



In [16]:
from mlxtend.frequent_patterns import association_rules
association_rules(apr, metric="confidence", min_threshold=0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cuticle Rehab),(Max AAA Batteries),0.005128,0.020513,0.005128,1.0,48.75,0.005023,inf
1,(Light Margarita),(Margarita Salt),0.005128,0.010256,0.005128,1.0,97.50,0.005076,inf


8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.

In [17]:
association_rules(apr, metric="lift", min_threshold=0.6).sort_values('lift', ascending=False)[['antecedents','consequents', 'lift']]

,antecedents,consequents,lift
10,(Light Margarita),(Margarita Salt),97.500
11,(Margarita Salt),(Light Margarita),97.500
6,(Cuticle Rehab),(Max AAA Batteries),48.750
7,(Max AAA Batteries),(Cuticle Rehab),48.750
8,(Italian Sweet Creme Creamer),(Rainbow Roll),48.750
9,(Rainbow Roll),(Italian Sweet Creme Creamer),48.750
12,(Margarita Salt),(The Original Margarita Mix),48.750
13,(The Original Margarita Mix),(Margarita Salt),48.750
2,(Organic Flax Seed),(Black Chia Seeds),32.500
3,(Black Chia Seeds),(Organic Flax Seed),32.500
